In [ ]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import altair as alt
import calitp_data_analysis.magics
import pandas as pd

from IPython.display import Markdown

from bus_service_utils import chart_utils
from calitp_data_analysis import calitp_color_palette as cp
from update_vars import GCS_FILE_PATH
from monthly_ridership_by_rtpa import get_percent_change

In [ ]:
# parameters cell for local
rtpa = "Southern California Association of Governments"

In [ ]:
#%%capture_parameters
rtpa

# {rtpa} 
## Monthly Ridership Trends

**Download the [zipped data](https://storage.googleapis.com/calitp-publish-data-analysis/ntd_monthly_ridership/).**

In [ ]:
df = pd.read_parquet(
    f"{GCS_FILE_PATH}ca_monthly_ridership_2023_October.parquet",
    filters = [[("RTPA", "==", rtpa)]]
)

In [ ]:
# find columns that are recent enough to plot
MIN_YEAR = 2018

not_id_cols = [c for c in df.columns if "/" in c]

recent_years = [
    c for c in not_id_cols if int(c.split("/")[1]) >= MIN_YEAR and 
    "pct" not in c
]

upt_cols = [
    c for c in recent_years if "change" not in c
]

change_cols = [c for c in recent_years if "change" in c]

In [ ]:
def sum_by_group(df: pd.DataFrame, group_cols: list) -> pd.DataFrame:
    """
    Since df is wide, use pivot_table() to sum up all
    the columns that show UPT.
    """
    grouped_df = df.pivot_table(
        index = group_cols, 
        values = recent_years, 
        aggfunc="sum"
    ).reset_index().reindex(columns = group_cols + recent_years)
    
    return grouped_df

def make_long(df: pd.DataFrame, group_cols: list, value_cols: list):
    df_long = df[group_cols + value_cols].melt(
        id_vars = group_cols, 
        value_vars = value_cols,
    )
    
    df_long = df_long.assign(
        variable = df_long.variable.str.replace("change_1yr_", "")
    )
    
    return df_long

In [ ]:
agency_cols = ["NTD ID", "Agency", "RTPA"]
mode_cols = ["Mode", "Mode_full", "RTPA"]
tos_cols = ["TOS", "TOS_full", "RTPA"]

by_agency = sum_by_group(df, agency_cols)
by_mode = sum_by_group(df, mode_cols)
by_tos = sum_by_group(df, tos_cols)

In [ ]:
def assemble_long_df(df: pd.DataFrame, group_cols: list) -> pd.DataFrame:
    """
    Need df to be long to make chart.
    Let's put raw UPT and change side-by-side.
    """
    df_raw = make_long(df, group_cols, upt_cols).rename(
        columns = {"value": "upt"})
    df_change = make_long(df, group_cols, change_cols).rename(
        columns = {"value": "change_1yr"})

    final = pd.merge(
        df_raw,
        df_change,
        on = group_cols + ["variable"],
        how = "left"
    )
    
    final = final.assign(
        year = final.variable.str.split("/", expand=True)[1],
        month = final.variable.str.split("/", expand=True)[0].str.zfill(2)
    )
    
    final = final.assign(
        year_month = final.year + "-" + final.month
    )
    
    return final

In [ ]:
by_agency_long = assemble_long_df(by_agency, agency_cols)
by_mode_long = assemble_long_df(by_mode, mode_cols)
by_tos_long = assemble_long_df(by_tos, tos_cols)

In [ ]:
LABELING_DICT = {
    "upt": "Unlinked Passenger Trips",
    "change_1yr": "Change in Unlinked Passenger Trips from Prior Year",
    "Mode_full": "Mode",
    "TOS_full": "Type of Service",
    "year_month": "Date"
}

def labeling(word: str) -> str:
    return chart_utils.labeling(word, LABELING_DICT)

In [ ]:
def make_line_chart(
    df: pd.DataFrame, 
    y_col: str,
    color_col: str,
) -> alt.Chart:
    df = df[df[y_col] > 0].dropna(subset = y_col)

    x_label = [i for i in df.year_month.unique() if 
               any(substring in i for substring in 
                   ["-01", "-06"])
              ]    
    chart = (alt.Chart(df)
         .mark_line()
         .encode(
             x = alt.X("year_month:O", 
                       axis=alt.Axis(values = x_label), 
                       title = "Date"
                      ),
             y = alt.Y(y_col, title = labeling(y_col)),
             color = alt.Color(color_col, 
                               scale = alt.Scale(
                                   range = cp.CALITP_CATEGORY_BRIGHT_COLORS + 
                                   cp.CALITP_CATEGORY_BOLD_COLORS
                               )),
             tooltip = ["year_month", y_col, color_col, "RTPA"]
         ).properties(width = 300, height = 150)
         .facet(color_col, columns=2)
         .resolve_scale(y="independent")
    ).properties(
        title = f"{labeling(y_col)} by {labeling(color_col)}"
    )
    
    chart = 
    
    return chart

In [ ]:
def make_bar_chart(
    df: pd.DataFrame, 
    y_col: str,
    color_col: str,
) -> alt.Chart:
    
    def short_label(word):
        shorten_dict = {
            "change_1yr": "Change",
        }
        return shorten_dict[word]
    
    # For change column, we are missing everything prior to 2023
    df = df[df[y_col] > 0].dropna(subset = y_col)
    
    x_label = [i for i in df.year_month.unique() if 
               any(substring in i for substring in 
                   ["-01", "-03", "-06", "-09"])
              ]
    
    chart = (alt.Chart(df)
         .mark_bar()
         .encode(
             x = alt.X("year_month:O", 
                       axis=alt.Axis(values = x_label), 
                       title = "Date"
                      ),
             y = alt.Y(y_col, title = short_label(y_col)),
             color = alt.Color(color_col, 
                              scale = alt.Scale(
                                   range = cp.CALITP_CATEGORY_BRIGHT_COLORS + 
                                   cp.CALITP_CATEGORY_BOLD_COLORS
                              )),
             tooltip = ["year_month", y_col, color_col, "RTPA"]
         ).properties(width = 300, height = 150)
         .facet(color_col, columns=2)
         .resolve_scale(x="shared", 
                        y="independent")
    ).properties(
        title = f"{labeling(y_col)} by {labeling(color_col)}"
    )
    
    return chart

## Transit Agency Breakdown

In [ ]:
make_line_chart(by_agency_long, y_col = "upt", color_col = "Agency")

In [ ]:
make_bar_chart(by_agency_long, y_col = "change_1yr", color_col = "Agency")

## Transit Mode Breakdown

In [ ]:
make_line_chart(by_mode_long, y_col = "upt", color_col = "Mode_full")

In [ ]:
make_bar_chart(by_mode_long, y_col = "change_1yr", color_col = "Mode_full")

## Type of Service Breakdown

In [ ]:
make_line_chart(by_tos_long, y_col = "upt", color_col = "TOS_full")

In [ ]:
make_bar_chart(by_tos_long, y_col = "change_1yr", color_col = "TOS_full")